In [4]:
%%capture

# !pip install snowflake-connector-python
!pip install "snowflake-connector-python[secure-local-storage,pandas]"

In [5]:
%%capture

!pip install snowflake-sqlalchemy

In [6]:
%%capture

!pip install xgboost

In [7]:
import os
import pickle
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import metrics

import sqlalchemy
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import *

import xgboost
from datetime import datetime, timedelta
import time  
import pytz    
tz_NY = pytz.timezone('Europe/London')

import snowflake_creds
import LOS_Preprocessing
from LOS_Preprocessing import preprocess_data

import warnings
warnings.filterwarnings('ignore')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/snowflake/sqlalchemy/base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [8]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
        account="vvkvijp-xf90326",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="HEALTHSCHEMA"
    ))

In [9]:
query = """

WITH BASE AS (

    SELECT CASE_ID,
           COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
           COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
           COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
           COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
           COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
           COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
           COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
           COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
           COALESCE(BED_GRADE,0) AS BED_GRADE,
           PATIENTID,
           COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
           COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
           COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
           COALESCE(AGE,'None') AS AGE,
           COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
           ADMISSION_DATE,
           DISCHARGE_DATE

    FROM HEALTHDB.HEALTHSCHEMA.SIMULATION_DATA_OCTOBER

),
    
BASE_WITH_FEATURES AS (

    SELECT *,
            MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
            DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,
            CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
            CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
            CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
            DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
    FROM BASE 

)    

SELECT * FROM BASE_WITH_FEATURES WHERE ADMISSION_DATE = CURRENT_DATE-2
"""

In [10]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

In [11]:
def insert_predictions_to_snowflake_table(data):
    import pandas
    import snowflake.connector
    from snowflake.connector.pandas_tools import pd_writer, write_pandas

    engine = create_engine(URL(
            account="vvkvijp-xf90326",
            user= snowflake_creds.USER_NAME,
            password= snowflake_creds.PASSWORD,
            role="ACCOUNTADMIN",
            warehouse="COMPUTE_WH",
            database="HEALTHDB",
            schema="HEALTHSCHEMA"
        ))  
    
    # Creating the logging table if not exists already
    table = 'TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI'
    
    # Inserting the data to snowflake logging table
    data.to_sql(table, engine, index=False, if_exists='append', method=pd_writer)
    return 'Success'

In [12]:
with engine.connect() as conn:
    
    # Loading the scoring data
    score_data = pd.DataFrame(pd.read_sql(query,conn))
    score_data.columns = [col.upper() for col in score_data.columns.tolist()]
    
    # Applying the preprocessing steps
    score_data_processed = LOS_Preprocessing.preprocess_data(score_data)
    
    # Applying feature selection
    final_feats = pd.read_pickle('MODEL_FEATS.pkl')
    score_data_final = check_n_create_model_features(score_data_processed,final_feats)
    
    # Getting the predictions
    model = xgboost.XGBRegressor()
    model.load_model('MODEL_XGB.model')
    score_data_final['PREDICTED_LOS'] = np.ceil(model.predict(score_data_final.drop('LOS',axis=1)))
    
    # Writing the dataframe to snowflake as a table
    score_data_final = score_data_final.reset_index()
    score_data_final.rename(columns={'index': 'CASE_ID'}, inplace=True)
    score_data_table = pd.merge(score_data,score_data_final,on='CASE_ID',how='left')
    status = insert_predictions_to_snowflake_table(score_data_table)

In [13]:
print(score_data.shape)
score_data.head()

(4787, 24)


,CASE_ID,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,PATIENTID,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,VISITORS_WITH_PATIENT,AGE,ADMISSION_DEPOSIT,ADMISSION_DATE,DISCHARGE_DATE,ADMISSION_MONTH,ADMISSION_DAY,ADMISSION_ILLNESS,ILLNESS_BEDGRADE,DEPARTMENT_ILLNESS,LOS
0,6,28,b,11,X,4,gynecology,R,F,3.0,91007,Emergency,Moderate,4,81-90,7574,2024-10-05,2024-11-07,Oct,Sat,Emergency-Moderate,Moderate-3.0,gynecology-Moderate,33
1,77,2,c,5,Z,3,gynecology,S,F,3.0,119622,Trauma,Extreme,5,71-80,4809,2024-10-05,2024-11-10,Oct,Sat,Trauma-Extreme,Extreme-3.0,gynecology-Extreme,36
2,78,12,a,9,Y,2,gynecology,R,B,1.0,119622,Urgent,Extreme,2,71-80,4509,2024-10-05,2024-10-09,Oct,Sat,Urgent-Extreme,Extreme-1.0,gynecology-Extreme,4
3,79,10,e,1,X,2,gynecology,S,E,3.0,53282,Trauma,Moderate,2,21-30,5968,2024-10-05,2024-10-09,Oct,Sat,Trauma-Moderate,Moderate-3.0,gynecology-Moderate,4
4,97,30,c,3,Z,6,gynecology,R,A,2.0,77607,Emergency,Extreme,2,81-90,4137,2024-10-05,2024-11-05,Oct,Sat,Emergency-Extreme,Extreme-2.0,gynecology-Extreme,31


In [14]:
print(score_data_processed.shape)
score_data_processed.head()

(4787, 100)


,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,VISITORS_WITH_PATIENT,ADMISSION_DEPOSIT,LOS,HOSPITAL_TYPE_CODE_a,HOSPITAL_TYPE_CODE_b,HOSPITAL_TYPE_CODE_c,HOSPITAL_TYPE_CODE_d,HOSPITAL_TYPE_CODE_e,HOSPITAL_TYPE_CODE_f,HOSPITAL_TYPE_CODE_g,CITY_CODE_HOSPITAL_1,CITY_CODE_HOSPITAL_2,CITY_CODE_HOSPITAL_3,CITY_CODE_HOSPITAL_4,CITY_CODE_HOSPITAL_5,CITY_CODE_HOSPITAL_6,CITY_CODE_HOSPITAL_7,CITY_CODE_HOSPITAL_9,CITY_CODE_HOSPITAL_10,CITY_CODE_HOSPITAL_11,CITY_CODE_HOSPITAL_13,HOSPITAL_REGION_CODE_X,HOSPITAL_REGION_CODE_Y,HOSPITAL_REGION_CODE_Z,DEPARTMENT_TB & Chest disease,DEPARTMENT_anesthesia,DEPARTMENT_gynecology,DEPARTMENT_radiotherapy,DEPARTMENT_surgery,WARD_TYPE_P,WARD_TYPE_Q,WARD_TYPE_R,WARD_TYPE_S,WARD_TYPE_T,WARD_TYPE_U,WARD_FACILITY_CODE_A,WARD_FACILITY_CODE_B,WARD_FACILITY_CODE_C,WARD_FACILITY_CODE_D,WARD_FACILITY_CODE_E,WARD_FACILITY_CODE_F,BED_GRADE_1.0,BED_GRADE_2.0,BED_GRADE_3.0,BED_GRADE_4.0,TYPE_OF_ADMISSION_Emergency,TYPE_OF_ADMISSION_Trauma,TYPE_OF_ADMISSION_Urgent,SEVERITY_OF_ILLNESS_Extreme,SEVERITY_OF_ILLNESS_Minor,SEVERITY_OF_ILLNESS_Moderate,AGE_0-10,AGE_20-Nov,AGE_21-30,AGE_31-40,AGE_41-50,AGE_51-60,AGE_61-70,AGE_71-80,AGE_81-90,AGE_91-100,ADMISSION_MONTH_Oct,ADMISSION_DAY_Sat,ADMISSION_ILLNESS_Emergency-Extreme,ADMISSION_ILLNESS_Emergency-Minor,ADMISSION_ILLNESS_Emergency-Moderate,ADMISSION_ILLNESS_Trauma-Extreme,ADMISSION_ILLNESS_Trauma-Minor,ADMISSION_ILLNESS_Trauma-Moderate,ADMISSION_ILLNESS_Urgent-Extreme,ADMISSION_ILLNESS_Urgent-Minor,ADMISSION_ILLNESS_Urgent-Moderate,ILLNESS_BEDGRADE_Extreme-1.0,ILLNESS_BEDGRADE_Extreme-2.0,ILLNESS_BEDGRADE_Extreme-3.0,ILLNESS_BEDGRADE_Extreme-4.0,ILLNESS_BEDGRADE_Minor-1.0,ILLNESS_BEDGRADE_Minor-2.0,ILLNESS_BEDGRADE_Minor-3.0,ILLNESS_BEDGRADE_Minor-4.0,ILLNESS_BEDGRADE_Moderate-1.0,ILLNESS_BEDGRADE_Moderate-2.0,ILLNESS_BEDGRADE_Moderate-3.0,ILLNESS_BEDGRADE_Moderate-4.0,DEPARTMENT_ILLNESS_TB & Chest disease-Extreme,DEPARTMENT_ILLNESS_TB & Chest disease-Minor,DEPARTMENT_ILLNESS_TB & Chest disease-Moderate,DEPARTMENT_ILLNESS_anesthesia-Extreme,DEPARTMENT_ILLNESS_anesthesia-Minor,DEPARTMENT_ILLNESS_anesthesia-Moderate,DEPARTMENT_ILLNESS_gynecology-Extreme,DEPARTMENT_ILLNESS_gynecology-Minor,DEPARTMENT_ILLNESS_gynecology-Moderate,DEPARTMENT_ILLNESS_radiotherapy-Extreme,DEPARTMENT_ILLNESS_radiotherapy-Minor,DEPARTMENT_ILLNESS_radiotherapy-Moderate,DEPARTMENT_ILLNESS_surgery-Extreme,DEPARTMENT_ILLNESS_surgery-Minor,DEPARTMENT_ILLNESS_surgery-Moderate
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,4,4,7574,33,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
77,3,5,4809,36,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
78,2,2,4509,4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,True,False,False,False,False,Fal

In [15]:
print(score_data_final.shape)
score_data_final.head()

(4787, 29)


,CASE_ID,ADMISSION_MONTH_Nov,ADMISSION_DAY_Tue,ADMISSION_MONTH_Oct,WARD_TYPE_R,CITY_CODE_HOSPITAL_7,WARD_TYPE_Q,ADMISSION_DAY_Sat,AGE_41-50,WARD_TYPE_S,ADMISSION_DAY_Wed,ADMISSION_MONTH_Sep,BED_GRADE_2,ILLNESS_BEDGRADE_Extreme-1,CITY_CODE_HOSPITAL_2,ADMISSION_DEPOSIT,TYPE_OF_ADMISSION_Trauma,AGE_51-60,ADMISSION_DAY_Mon,VISITORS_WITH_PATIENT,ADMISSION_DAY_Sun,CITY_CODE_PATIENT_8,ADMISSION_DAY_Thu,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,SEVERITY_OF_ILLNESS_Minor,AGE_71-80,AGE_31-40,LOS,PREDICTED_LOS
0,6,NaN,NaN,True,True,False,False,True,False,False,0,0,0,0,False,7574,False,False,0,4,0,0,0,4,False,False,False,33,33.0
1,77,NaN,NaN,True,False,False,False,True,False,True,0,0,0,0,False,4809,True,False,0,5,0,0,0,3,False,True,False,36,38.0
2,78,NaN,NaN,True,True,False,False,True,False,False,0,0,0,0,False,4509,False,False,0,2,0,0,0,2,False,True,False,4,24.0
3,79,NaN,NaN,True,False,False,False,True,False,True,0,0,0,0,False,5968,True,False,0,2,0,0,0,2,False,False,False,4,26.0
4,97,NaN,NaN,True,True,False,False,True,False,False,0,0,0,0,False,4137,False,False,0,2,0,0,0,6,False,False,False,31,23.0


In [16]:
print(score_data_table.shape)
score_data_table.head()

(4787, 52)


,CASE_ID,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_x,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,PATIENTID,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,VISITORS_WITH_PATIENT_x,AGE,ADMISSION_DEPOSIT_x,ADMISSION_DATE,DISCHARGE_DATE,ADMISSION_MONTH,ADMISSION_DAY,ADMISSION_ILLNESS,ILLNESS_BEDGRADE,DEPARTMENT_ILLNESS,LOS_x,ADMISSION_MONTH_Nov,ADMISSION_DAY_Tue,ADMISSION_MONTH_Oct,WARD_TYPE_R,CITY_CODE_HOSPITAL_7,WARD_TYPE_Q,ADMISSION_DAY_Sat,AGE_41-50,WARD_TYPE_S,ADMISSION_DAY_Wed,ADMISSION_MONTH_Sep,BED_GRADE_2,ILLNESS_BEDGRADE_Extreme-1,CITY_CODE_HOSPITAL_2,ADMISSION_DEPOSIT_y,TYPE_OF_ADMISSION_Trauma,AGE_51-60,ADMISSION_DAY_Mon,VISITORS_WITH_PATIENT_y,ADMISSION_DAY_Sun,CITY_CODE_PATIENT_8,ADMISSION_DAY_Thu,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_y,SEVERITY_OF_ILLNESS_Minor,AGE_71-80,AGE_31-40,LOS_y,PREDICTED_LOS
0,6,28,b,11,X,4,gynecology,R,F,3.0,91007,Emergency,Moderate,4,81-90,7574,2024-10-05,2024-11-07,Oct,Sat,Emergency-Moderate,Moderate-3.0,gynecology-Moderate,33,NaN,NaN,True,True,False,False,True,False,False,0,0,0,0,False,7574,False,False,0,4,0,0,0,4,False,False,False,33,33.0
1,77,2,c,5,Z,3,gynecology,S,F,3.0,119622,Trauma,Extreme,5,71-80,4809,2024-10-05,2024-11-10,Oct,Sat,Trauma-Extreme,Extreme-3.0,gynecology-Extreme,36,NaN,NaN,True,False,False,False,True,False,True,0,0,0,0,False,4809,True,False,0,5,0,0,0,3,False,True,False,36,38.0
2,78,12,a,9,Y,2,gynecology,R,B,1.0,119622,Urgent,Extreme,2,71-80,4509,2024-10-05,2024-10-09,Oct,Sat,Urgent-Extreme,Extreme-1.0,gynecology-Extreme,4,NaN,NaN,True,True,False,False,True,False,False,0,0,0,0,False,4509,False,False,0,2,0,0,0,2,False,True,False,4,24.0
3,79,10,e,1,X,2,gynecology,S,E,3.0,53282,Trauma,Moderate,2,21-30,5968,2024-10-05,2024-10-09,Oct,Sat,Trauma-Moderate,Moderate-3.0,gynecology-Moderate,4,NaN,NaN,True,False,False,False,True,False,True,0,0,0,0,False,5968,True,False,0,2,0,0,0,2,False,False,False,4,26.0
4,97,30,c,3,Z,6,gynecology,R,A,2.0,77607,Emergency,Extreme,2,81-90,4137,2024-10-05,2024-11-05,Oct,Sat,Emergency-Extreme,Extreme-2.0,gynecology-Extreme,31,NaN,NaN,True,True,False,False,True,False,False,0,0,0,0,False,4137,False,False,0,2,0,0,0,6,False,False,False,31,23.0


# Scoring Function for Deployment:

In [17]:
query = """

WITH BASE AS (

    SELECT CASE_ID,
           COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
           COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
           COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
           COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
           COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
           COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
           COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
           COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
           COALESCE(BED_GRADE,0) AS BED_GRADE,
           PATIENTID,
           COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
           COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
           COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
           COALESCE(AGE,'None') AS AGE,
           COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
           ADMISSION_DATE,
           DISCHARGE_DATE

    FROM HEALTHDB.HEALTHSCHEMA.SIMULATION_DATA_OCTOBER

),
    
BASE_WITH_FEATURES AS (

    SELECT *,
            MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
            DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,
            CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
            CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
            CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
            DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
    FROM BASE 

)    

SELECT * FROM BASE_WITH_FEATURES WHERE ADMISSION_DATE = CURRENT_DATE-2
"""

In [18]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

In [19]:
def insert_predictions_to_snowflake_table(data):
    import pandas
    import snowflake.connector
    from snowflake.connector.pandas_tools import pd_writer, write_pandas

    engine = create_engine(URL(
            account="vvkvijp-xf90326",
            user= snowflake_creds.USER_NAME,
            password= snowflake_creds.PASSWORD,
            role="ACCOUNTADMIN",
            warehouse="COMPUTE_WH",
            database="HEALTHDB",
            schema="HEALTHSCHEMA"
        ))  
    
    # Creating the logging table if not exists already
    table = 'TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI'
    
    # Inserting the data to snowflake logging table
    data.to_sql(table, engine, index=False, if_exists='append', method=pd_writer)
    return 'Success'

In [20]:
# def send_status_mail(mail_string):
#     import mail_creds
#     import smtplib
#     from email.mime.multipart import MIMEMultipart
#     from email.mime.text import MIMEText
#     from email.mime.multipart import MIMEMultipart
#     from email.mime.base import MIMEBase
#     from email.mime.text import MIMEText
#     from email.utils import formatdate
#     from email import encoders
    
#     subject = 'Patient LOS Prediction - STATUS MAIL'
#     mail_content = mail_string

#     username= mail_creds.MAIL_ID
#     password= mail_creds.APP_PASSWORD
#     send_from = mail_creds.MAIL_ID
#     send_to = mail_creds.MAIL_ID
#     Cc = ''
#     msg = MIMEMultipart()
#     msg['From'] = send_from
#     msg['To'] = send_to
#     msg['Cc'] = Cc
#     msg['Date'] = formatdate(localtime = True)
#     msg['Subject'] = subject
#     msg.attach(MIMEText(mail_content, 'plain'))
#     smtp = smtplib.SMTP('smtp.gmail.com',587)
#     smtp.ehlo()
#     smtp.starttls()
#     smtp.login(username,password)
#     smtp.sendmail(send_from, send_to.split(',') + msg['Cc'].split(','), msg.as_string())
#     smtp.quit()    

In [21]:
# Alternative Email Notification

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_status_mail(mail_string):
    subject = 'Patient LOS Prediction - STATUS MAIL'
    mail_content = mail_string
    username = 'oluwanishola.ayeotan@gmail.com'  # Your Gmail address
    password = 'xjjcrqxemyrodkfx'  # Use the generated App Password
    send_from = username
    send_to = username

    # Create the email content
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = send_to
    msg['Subject'] = subject
    msg.attach(MIMEText(mail_content, 'plain'))

    # Sending the email via Gmail's SMTP server
    try:
        smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465)  # Use SSL for security
        smtp.login(username, password)
        smtp.sendmail(send_from, send_to, msg.as_string())
        smtp.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")


In [22]:
# def LOS_MODEL_DEPLOYMENT():
#     try:
#         import pickle
#         import pandas as pd
#         import numpy as np
#         import sqlalchemy
#         import snowflake.connector
#         from sqlalchemy import create_engine

#         import xgboost as xgb
#         from datetime import datetime, timedelta
#         import time  
#         import pytz    
#         tz_NY = pytz.timezone('Europe/London')

#         import warnings
#         warnings.filterwarnings('ignore')   
        
#         import snowflake_creds
#         import LOS_Preprocessing
#         from LOS_Preprocessing import preprocess_data
        
#         # Creating the connection engine (way 1)
#         engine = create_engine(URL(
#                 account="vvkvijp-xf90326",
#                 user= snowflake_creds.USER_NAME,
#                 password= snowflake_creds.PASSWORD,
#                 role="ACCOUNTADMIN",
#                 warehouse="COMPUTE_WH",
#                 database="HEALTHDB",
#                 schema="HEALTHSCHEMA"
#             ))
        
#         mail_list = []
        
#         with engine.connect() as conn:
    
#             # Loading the scoring data
#             score_data = pd.DataFrame(pd.read_sql(query,conn))
#             score_data.columns = [col.upper() for col in score_data.columns.tolist()]
#             mail_list.append('STEP-1: Loading Data complete')

#             # Applying the preprocessing steps
#             score_data_processed = LOS_Preprocessing.preprocess_data(score_data)
#             mail_list.append('STEP-2: Applying the Preprocessing Steps complete')

#             # Applying feature selection
#             final_feats = pd.read_pickle('MODEL_FEATS.pkl')
#             score_data_final = check_n_create_model_features(score_data_processed,final_feats)
#             mail_list.append('STEP-3: Applying the Feature Selection Steps complete')

#             # Getting the predictions
#             model = xgboost.XGBRegressor()
#             model.load_model('MODEL_XGB.model')
#             score_data_final['PREDICTED_LOS'] = np.ceil(model.predict(score_data_final.drop('LOS',axis=1)))
#             mail_list.append('STEP-4: Getting the Predictions complete')

#             # Writing the dataframe to snowflake as a table
#             score_data_final = score_data_final.reset_index()
#             score_data_final.rename(columns={'index': 'CASE_ID'}, inplace=True)
#             score_data_table = pd.merge(score_data,score_data_final,on='CASE_ID',how='left')
#             status = insert_predictions_to_snowflake_table(score_data_table)
#             mail_list.append('STEP-5: Writing the data to snowflake complete')
            
#             # Creating the mail body
#             mail_string = ",\n ".join(map(str, mail_list))
#             send_status_mail(mail_string)
#             print('Success')
    
#     except Exception as e:
#         mail_content = f'Schedule failed due to error: {str(e)}'
#         send_status_mail(mail_content)
#         print(f'Failed with error: {str(e)}')


In [23]:
def LOS_MODEL_DEPLOYMENT():
    try:
        import pickle
        import pandas as pd
        import numpy as np
        import sqlalchemy
        import snowflake.connector
        from sqlalchemy import create_engine
        import xgboost as xgb
        from datetime import datetime, timedelta
        import time  
        import pytz    
        tz_NY = pytz.timezone('Europe/London')

        import warnings
        warnings.filterwarnings('ignore')   
        
        import snowflake_creds
        import LOS_Preprocessing
        from LOS_Preprocessing import preprocess_data
        
        mail_list = []
        
        try:
            # Creating the connection engine
            engine = create_engine(URL(
                account="vvkvijp-xf90326",
                user=snowflake_creds.USER_NAME,
                password=snowflake_creds.PASSWORD,
                role="ACCOUNTADMIN",
                warehouse="COMPUTE_WH",
                database="HEALTHDB",
                schema="HEALTHSCHEMA"
            ))
            mail_list.append('STEP-0: Snowflake connection established')
            print("STEP-0: Snowflake connection established.")
        except Exception as e:
            mail_list.append(f"Error in Snowflake connection: {str(e)}")
            send_status_mail(",\n ".join(map(str, mail_list)))
            print(f"Error in Snowflake connection: {str(e)}")
            raise e
        
        with engine.connect() as conn:
            
            try:
                # Loading the scoring data
                score_data = pd.DataFrame(pd.read_sql(query, conn))
                score_data.columns = [col.upper() for col in score_data.columns.tolist()]
                mail_list.append('STEP-1: Loading Data complete')
                print("STEP-1: Loading Data complete.")
            except Exception as e:
                mail_list.append(f"Error in loading data: {str(e)}")
                send_status_mail(",\n ".join(map(str, mail_list)))
                print(f"Error in loading data: {str(e)}")
                raise e
            
            try:
                # Applying the preprocessing steps
                score_data_processed = LOS_Preprocessing.preprocess_data(score_data)
                mail_list.append('STEP-2: Applying the Preprocessing Steps complete')
                print("STEP-2: Preprocessing applied successfully.")
            except Exception as e:
                mail_list.append(f"Error in preprocessing data: {str(e)}")
                send_status_mail(",\n ".join(map(str, mail_list)))
                print(f"Error in preprocessing data: {str(e)}")
                raise e
            
            try:
                # Applying feature selection
                final_feats = pd.read_pickle('MODEL_FEATS.pkl')
                score_data_final = check_n_create_model_features(score_data_processed, final_feats)
                mail_list.append('STEP-3: Applying the Feature Selection Steps complete')
                print("STEP-3: Feature selection applied.")
            except Exception as e:
                mail_list.append(f"Error in feature selection: {str(e)}")
                send_status_mail(",\n ".join(map(str, mail_list)))
                print(f"Error in feature selection: {str(e)}")
                raise e

            try:
                # Getting the predictions
                model = xgb.XGBRegressor()
                model.load_model('MODEL_XGB.model')
                score_data_final['PREDICTED_LOS'] = np.ceil(model.predict(score_data_final.drop('LOS', axis=1)))
                mail_list.append('STEP-4: Getting the Predictions complete')
                print("STEP-4: Predictions generated.")
            except Exception as e:
                mail_list.append(f"Error in model prediction: {str(e)}")
                send_status_mail(",\n ".join(map(str, mail_list)))
                print(f"Error in model prediction: {str(e)}")
                raise e
            
            try:
                # Writing the dataframe to Snowflake as a table
                score_data_final = score_data_final.reset_index()
                score_data_final.rename(columns={'index': 'CASE_ID'}, inplace=True)
                score_data_table = pd.merge(score_data, score_data_final, on='CASE_ID', how='left')
                status = insert_predictions_to_snowflake_table(score_data_table)
                mail_list.append('STEP-5: Writing the data to Snowflake complete')
                print("STEP-5: Data written to Snowflake.")
            except Exception as e:
                mail_list.append(f"Error in writing to Snowflake: {str(e)}")
                send_status_mail(",\n ".join(map(str, mail_list)))
                print(f"Error in writing to Snowflake: {str(e)}")
                raise e
        
        # Creating the mail body
        mail_string = ",\n ".join(map(str, mail_list))
        send_status_mail(mail_string)
        print('Success')

    except Exception as e:
        mail_content = f'Schedule failed: {str(e)}'
        send_status_mail(mail_content)
        print(f'Schedule failed: {str(e)}')

In [24]:
LOS_MODEL_DEPLOYMENT()

STEP-0: Snowflake connection established.
STEP-1: Loading Data complete.
STEP-2: Preprocessing applied successfully.
STEP-3: Feature selection applied.
STEP-4: Predictions generated.
STEP-5: Data written to Snowflake.
Email sent successfully!
Success


In [ ]:
%%capture
pip install schedule

In [ ]:
import schedule
import time
from datetime import datetime
import pytz

def LOS_MODEL_DEPLOYMENT():
    try:
        print(f"Running LOS_MODEL_DEPLOYMENT at {datetime.now(pytz.timezone('Europe/London'))}")
        # Your function logic goes here
        # ...
        print("LOS_MODEL_DEPLOYMENT completed.")
    except Exception as e:
        print(f"Error during LOS_MODEL_DEPLOYMENT: {e}")

def run_scheduler():
    # Schedule tasks for the specific times in Europe/London timezone
    schedule.every().day.at("10:10").do(LOS_MODEL_DEPLOYMENT)
    schedule.every().day.at("17:17").do(LOS_MODEL_DEPLOYMENT)

    print("Scheduler started...")
    
    while True:
        # Run the scheduled jobs
        schedule.run_pending()
        time.sleep(1)  # Wait for one second before checking again

if __name__ == "__main__":
    run_scheduler()


In [ ]:
import time
from datetime import datetime    
import pytz    
tz_NY = pytz.timezone('Europe/London')   

# scheduled hours in 24-hour format (ensure two digits for hour and minute)
hours_list = ["10:10", "17:17"]

while True:
    tz_NY = pytz.timezone('Europe/London')   
    now = datetime.now(tz_NY)  # gets current datetime

    # Zero-pads hour and minute to two digits
    hour = f"{now.hour:02d}"
    minute = f"{now.minute:02d}"
    current_time = f"{hour}:{minute}"  # combines current hour and minute

    # checks if current time is in the hours list
    if current_time in hours_list:
        LOS_MODEL_DEPLOYMENT()
    
    time.sleep(60)  # waits a minute until it repeats


In [25]:
print(datetime.now())

2024-10-07 20:53:26.738367


In [26]:
tz_NY = pytz.timezone('Europe/London') 
print(datetime.now(tz_NY))

2024-10-07 21:53:27.334529+01:00


# Live Data Scoring Simulation:

In [55]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
        account="vvkvijp-xf90326",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="HEALTHSCHEMA"
    ))

In [56]:
def scoring_query(a):
    query_sim = f"""

    WITH BASE AS (

        SELECT CASE_ID,
               COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
               COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
               COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
               COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
               COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
               COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
               COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
               COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
               COALESCE(BED_GRADE,0) AS BED_GRADE,
               PATIENTID,
               COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
               COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
               COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
               COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
               COALESCE(AGE,'None') AS AGE,
               COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
               ADMISSION_DATE,
               DISCHARGE_DATE

        FROM HEALTHDB.HEALTHSCHEMA.SIMULATION_DATA_OCTOBER

    ),

    BASE_WITH_FEATURES AS (

        SELECT *,
                MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
                DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,
                CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
                CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
                CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
                DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
        FROM BASE 

    )    

    SELECT * FROM BASE_WITH_FEATURES WHERE ADMISSION_DATE = CURRENT_DATE-{a}
    """
    return query_sim

In [57]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

In [58]:
def insert_predictions_to_snowflake_table(data):
    import pandas
    import snowflake.connector
    from snowflake.connector.pandas_tools import pd_writer, write_pandas

    engine = create_engine(URL(
            account="vvkvijp-xf90326",
            user= snowflake_creds.USER_NAME,
            password= snowflake_creds.PASSWORD,
            role="ACCOUNTADMIN",
            warehouse="COMPUTE_WH",
            database="HEALTHDB",
            schema="HEALTHSCHEMA"
        ))  
    
    # Creating the logging table if not exists already
    table = 'TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI'
    
    # Inserting the data to snowflake logging table
    data.to_sql(table, engine, index=False, if_exists='append', method=pd_writer)
    return 'Success'

In [59]:
# def send_status_mail(mail_string):
#     import mail_creds
#     import smtplib
#     from email.mime.multipart import MIMEMultipart
#     from email.mime.text import MIMEText
#     from email.mime.multipart import MIMEMultipart
#     from email.mime.base import MIMEBase
#     from email.mime.text import MIMEText
#     from email.utils import formatdate
#     from email import encoders
    
#     subject = 'Patient LOS Prediction - STATUS MAIL'
#     mail_content = mail_string

#     username= mail_creds.MAIL_ID
#     password= mail_creds.APP_PASSWORD
#     send_from = mail_creds.MAIL_ID
#     send_to = mail_creds.MAIL_ID
#     Cc = ''
#     msg = MIMEMultipart()
#     msg['From'] = send_from
#     msg['To'] = send_to
#     msg['Cc'] = Cc
#     msg['Date'] = formatdate(localtime = True)
#     msg['Subject'] = subject
#     msg.attach(MIMEText(mail_content, 'plain'))
#     smtp = smtplib.SMTP('smtp.gmail.com',587)
#     smtp.ehlo()
#     smtp.starttls()
#     smtp.login(username,password)
#     smtp.sendmail(send_from, send_to.split(',') + msg['Cc'].split(','), msg.as_string())
#     smtp.quit()    

In [60]:
# Alternative Email Notification

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_status_mail(mail_string):
    subject = 'Patient LOS Prediction - STATUS MAIL'
    mail_content = mail_string
    username = 'oluwanishola.ayeotan@gmail.com'  # Your Gmail address
    password = 'xjjcrqxemyrodkfx'  # Use the generated App Password
    send_from = username
    send_to = username

    # Create the email content
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = send_to
    msg['Subject'] = subject
    msg.attach(MIMEText(mail_content, 'plain'))

    # Sending the email via Gmail's SMTP server
    try:
        smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465)  # Use SSL for security
        smtp.login(username, password)
        smtp.sendmail(send_from, send_to, msg.as_string())
        smtp.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")


In [61]:
def LOS_MODEL_DEPLOYMENT_SIM(x):
    try:
        import pickle
        import pandas as pd
        import numpy as np
        import sqlalchemy
        import snowflake.connector
        from sqlalchemy import create_engine

        import xgboost as xgb
        from datetime import datetime, timedelta
        import time  
        import pytz    
        tz_NY = pytz.timezone('Europe/London')

        import warnings
        warnings.filterwarnings('ignore')   
        
        import snowflake_creds
        import LOS_Preprocessing
        from LOS_Preprocessing import preprocess_data
        
        # Creating the connection engine (way 1)
        engine = create_engine(URL(
                account="vvkvijp-xf90326",
                user= snowflake_creds.USER_NAME,
                password= snowflake_creds.PASSWORD,
                role="ACCOUNTADMIN",
                warehouse="COMPUTE_WH",
                database="HEALTHDB",
                schema="HEALTHSCHEMA"
            ))
        
        mail_list = []
        
        with engine.connect() as conn:
    
            # Loading the scoring data
#             print(scoring_query(x))
            score_data = pd.DataFrame(pd.read_sql(scoring_query(x),conn))
            score_data.columns = [col.upper() for col in score_data.columns.tolist()]
            print(score_data['ADMISSION_DATE'].max())
            mail_list.append('STEP-1: Loading Data complete')

            # Applying the preprocessing steps
            score_data_processed = LOS_Preprocessing.preprocess_data(score_data)
            mail_list.append('STEP-2: Applying the Preprocessing Steps complete')

            # Applying feature selection
            final_feats = pd.read_pickle('MODEL_FEATS.pkl')
            score_data_final = check_n_create_model_features(score_data_processed,final_feats)
            mail_list.append('STEP-3: Applying the Feature Selection Steps complete')

            # Getting the predictions
            model = xgboost.XGBRegressor()
            model.load_model('MODEL_XGB.model')
            score_data_final['PREDICTED_LOS'] = np.ceil(model.predict(score_data_final.drop('LOS',axis=1)))
            mail_list.append('STEP-4: Getting the Predictions complete')

            # Writing the dataframe to snowflake as a table
            score_data_final = score_data_final.reset_index()
            score_data_table = pd.merge(score_data,score_data_final,on='CASE_ID',how='left')
            status = insert_predictions_to_snowflake_table(score_data_table)
            mail_list.append('STEP-5: Writing the data to snowflake complete')
            
            # Creating the mail body
            mail_string = ",\n ".join(map(str, mail_list))
            send_status_mail(mail_string)
            print('Success')
    
    except Exception as e:
            mail_content = f'Schedule failed: {str(e)}'
            send_status_mail(mail_content)
            print(f'Schedule failed: {str(e)}')

In [64]:
def LOS_MODEL_DEPLOYMENT_SIM(x):
    try:
        import pickle
        import pandas as pd
        import numpy as np
        import sqlalchemy
        import snowflake.connector
        from sqlalchemy import create_engine
        import xgboost as xgb
        from datetime import datetime, timedelta
        import pytz
        tz_NY = pytz.timezone('Europe/London')

        import warnings
        warnings.filterwarnings('ignore')
        
        import snowflake_creds
        import LOS_Preprocessing
        from LOS_Preprocessing import preprocess_data
        
        # Creating the connection engine
        engine = create_engine(URL(
                account="vvkvijp-xf90326",
                user= snowflake_creds.USER_NAME,
                password= snowflake_creds.PASSWORD,
                role="ACCOUNTADMIN",
                warehouse="COMPUTE_WH",
                database="HEALTHDB",
                schema="HEALTHSCHEMA"
            ))
        
        mail_list = []
        
        with engine.connect() as conn:
    
            # Simulate loading scoring data for the given past day (e.g., simulate x days before today)
            print(f"Simulating for {x} days back (simulating for date: CURRENT_DATE - {x})")
            score_data = pd.DataFrame(pd.read_sql(scoring_query(x), conn))
            score_data.columns = [col.upper() for col in score_data.columns.tolist()]
            print(f"Max Admission Date in simulated data: {score_data['ADMISSION_DATE'].max()}")
            mail_list.append('STEP-1: Loading Data complete')

            # Applying the preprocessing steps
            score_data_processed = LOS_Preprocessing.preprocess_data(score_data)
            mail_list.append('STEP-2: Preprocessing applied successfully.')

            # Applying feature selection
            final_feats = pd.read_pickle('MODEL_FEATS.pkl')
            score_data_final = check_n_create_model_features(score_data_processed, final_feats)
            mail_list.append('STEP-3: Feature Selection applied.')

            # Getting the predictions
            model = xgboost.XGBRegressor()
            model.load_model('MODEL_XGB.model')
            score_data_final['PREDICTED_LOS'] = np.ceil(model.predict(score_data_final.drop('LOS', axis=1)))
            mail_list.append('STEP-4: Predictions generated.')

            # Writing the dataframe to Snowflake as a table
            score_data_final = score_data_final.reset_index()
            score_data_final.rename(columns={'index': 'CASE_ID'}, inplace=True)  # rename index for the merge
            score_data_table = pd.merge(score_data, score_data_final, on='CASE_ID', how='left')
            status = insert_predictions_to_snowflake_table(score_data_table)
            mail_list.append('STEP-5: Data written to Snowflake.')

            # Creating the mail body
            mail_string = ",\n ".join(map(str, mail_list))
            send_status_mail(mail_string)
            print('Success')

    except Exception as e:
        mail_content = f'Schedule failed: {str(e)}'
        send_status_mail(mail_content)
        print(f'Schedule failed: {str(e)}')


In [62]:
LOS_MODEL_DEPLOYMENT_SIM(2)

2024-10-05
Email sent successfully!
Success


In [63]:
# Scheduling the notebook (for live scoring simulation)

import time
from datetime import datetime    
import pytz    
tz_NY = pytz.timezone('Europe/London')   
 

# scheduled hours in 24-hour format
hours_list = ["10:10","17:17"]
 
for i in range(1,4):
#     tz_NY = pytz.timezone('Asia/Kolkata')   
#     now = datetime.now(tz_NY)  # gets current datetime

#     hour = str(now.hour) # gets current hour
#     minute = str(now.minute) # gets current minute
#     current_time = f"{hour}:{minute}" # combines current hour and minute

#     # checks if current time is in the hours list
#     if current_time in hours_list:
#         LOS_MODEL_DEPLOYMENT()
    
    # Run the scoring script
    LOS_MODEL_DEPLOYMENT_SIM(i)
    
    #time.sleep(60) # waits a minute until it repeats


2024-10-06
Email sent successfully!
Schedule failed: 100071 (22000): Failed to cast variant value 0 to BOOLEAN
2024-10-05
Email sent successfully!
Success
2024-10-04
Email sent successfully!
Schedule failed: 100071 (22000): Failed to cast variant value 0 to BOOLEAN
